## **Anexo proyecto final Bootcamp**

In [1]:
import pandas as pd

# Cargar el archivo
df = pd.read_excel('https://github.com/luisalbertodavidbetancur/municipios-antioquia/raw/refs/heads/main/MUNICIPIOS_ANTIOQUIA_REORGANIZADO%20(1).xlsx')


### **Limpieza de Columnas**

In [2]:
# Limpiar nombres de columnas (por si tienen espacios)
df.columns = df.columns.str.strip().str.lower()

# Detectar columnas que contienen el símbolo %
columnas_porcentaje = [col for col in df.columns if '%' in col or 'porcentaje' in col or 'irca' in col]

# Limpiar cada una de esas columnas
for col in columnas_porcentaje:
    df[col] = (
        df[col].astype(str)
        .str.extract(r'([\d\.,]+)')[0]     # Extrae solo el número
        .str.replace(',', '.', regex=False)
        .astype(float)
    )



### **Limpiar y reemplazar porcentajes**

In [3]:
# Función para limpiar valores de porcentaje
def limpiar_porcentaje(valor):
    if pd.isnull(valor):
        return None
    valor_str = str(valor).strip().replace('%', '').replace(',', '.')
    try:
        num = float(valor_str)
        if num <= 1:
            num *= 100
        return round(num, 2)
    except:
        return None

# Buscar columnas con "porcentaje" o "%"
cols_porcentaje = [col for col in df.columns if 'porcentaje' in col.lower() or '%' in col]

# Limpiar y reemplazar los valores directamente en esas columnas
for col in cols_porcentaje:
    df[col] = df[col].apply(limpiar_porcentaje)



### **Reemplazar Datos Textuales por Números**

In [4]:
import re

# Función para extraer número de horas de un texto como "24H/D", "23 h", "12 h/día"
def limpiar_horas(valor):
    if pd.isnull(valor):
        return None
    valor_str = str(valor).lower()
    match = re.search(r'(\d+)', valor_str)
    if match:
        return int(match.group(1))
    return None

# Buscar columnas candidatas con palabras clave relacionadas a horas
cols_horas = [col for col in df.columns if any(kw in col.lower() for kw in ['hora', 'continuidad'])]

# Aplicar limpieza
for col in cols_horas:
    df[col] = df[col].apply(limpiar_horas)



### **Estandarizar valores tipo Sí/No**

In [5]:
# Función para estandarizar valores tipo Sí/No
def limpiar_sino(valor):
    if pd.isnull(valor):
        return None
    valor_str = str(valor).strip().lower()
    if valor_str in ['si', 'sí']:
        return 'Sí'
    elif valor_str == 'no':
        return 'No'
    else:
        return None  # o puedes devolver el valor original si prefieres

# Detectar columnas candidatas: las que tengan pocos valores únicos y contengan "sí" o "no"
candidatas = []
for col in df.columns:
    valores = df[col].dropna().astype(str).str.lower().unique()
    if any(v in ['si', 'sí', 'no'] for v in valores) and len(valores) <= 5:
        candidatas.append(col)

# Aplicar la limpieza
for col in candidatas:
    df[col] = df[col].apply(limpiar_sino)



### **Convertir números a porcentajes**

In [6]:
columnas_porcentaje = ['calidad del agua urbana - irca (%)', 'cobertura acueducto rural (%)','cobertura acueducto urbano (%)', 'cobertura alcantarillado rural (%)', 'cobertura alcantarillado urbano (%)', 'cobertura aseo rural (%)', 'cobertura aseo urbano (%)', 'porcentaje de aguas residuales tratadas (%)', 'porcentaje de aprovechamiento (%)' ]  # reemplaza con las que necesites


def format_pct(x):
    try:
        num = float(x)              # si x es convertible a float
        return f"{num:.2f}%"
    except:
        return x                    # si no, lo dejamos igual

for col in columnas_porcentaje:
    df[col] = df[col].apply(format_pct)

In [7]:
df.head()

,municipio,aprovechamiento viable (sí/no),calidad del agua urbana - irca (%),cobertura acueducto rural (%),cobertura acueducto urbano (%),cobertura alcantarillado rural (%),cobertura alcantarillado urbano (%),cobertura aseo rural (%),cobertura aseo urbano (%),continuidad acueducto urbano (hora/día),disposición final adecuada (sí/no),población rural (número de habitantes),población urbana (número de habitantes),porcentaje de aguas residuales tratadas (%),porcentaje de aprovechamiento (%)
0,ABEJORRAL,Sí,0.50%,47.00%,98.00%,44.52%,95.00%,8.59%,100.00%,24,Sí,12237,8365.0,60.00%,23.15%
1,ABRIAQUI,Sí,10.19%,40.00%,100.00%,4.90%,100.00%,0.00%,100.00%,24,Sí,1788,625.0,98.00%,20.00%
2,ALEJANDRIA,Sí,0.75%,78.50%,100.00%,0.00%,98.00%,0.00%,100.00%,24,Sí,1700,3702.0,98.00%,64.00%
3,AMAGA,No,0.69%,92.36%,100.00%,87.50%,96.00%,88.00%,100.00%,23,No,19404,16936.0,0.00%,0.00%
4,AMALFI,Sí,0.00%,12.60%,100.00%,2.00%,93.08%,39.00%,100.00%,24,Sí,10981,27846.0,86.00%,18.09%


In [8]:
df[df.isna().all(axis=1)]

,municipio,aprovechamiento viable (sí/no),calidad del agua urbana - irca (%),cobertura acueducto rural (%),cobertura acueducto urbano (%),cobertura alcantarillado rural (%),cobertura alcantarillado urbano (%),cobertura aseo rural (%),cobertura aseo urbano (%),continuidad acueducto urbano (hora/día),disposición final adecuada (sí/no),población rural (número de habitantes),población urbana (número de habitantes),porcentaje de aguas residuales tratadas (%),porcentaje de aprovechamiento (%)


### **Anexar datos a columnas que se encuentran vacias**

In [9]:
df.loc[47, 'cobertura aseo rural (%)'] = 0
df.loc[60, 'cobertura aseo rural (%)'] = 30
df.loc[60, 'disposición final adecuada (sí/no)'] = 'Sí'
df.loc[75, 'porcentaje de aguas residuales tratadas (%)'] = 0
df.loc[79, 'porcentaje de aguas residuales tratadas (%)'] = 99.69
df.loc[79, 'porcentaje de aprovechamiento (%)'] = 12.13
df.loc[80, 'porcentaje de aguas residuales tratadas (%)'] = 99

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 15 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   municipio                                    125 non-null    object 
 1   aprovechamiento viable (sí/no)               125 non-null    object 
 2   calidad del agua urbana - irca (%)           125 non-null    object 
 3   cobertura acueducto rural (%)                125 non-null    object 
 4   cobertura acueducto urbano (%)               125 non-null    object 
 5   cobertura alcantarillado rural (%)           125 non-null    object 
 6   cobertura alcantarillado urbano (%)          125 non-null    object 
 7   cobertura aseo rural (%)                     125 non-null    object 
 8   cobertura aseo urbano (%)                    125 non-null    object 
 9   continuidad acueducto urbano (hora/día)      125 non-null    int64  
 10  di